In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def prepare_data():
    """ 
    準備資料
    Returns:
        X_train(ndarray): 訓練資料 (50000.32.32.3)
        X_test(ndarray): 測試資料 (10000.32.32.3)
        y_train(ndarray): 將訓練資料改為One-hot encoding後的標籤 (50000,10)
        y_train(ndarray): 將測試資料改為One-hot encoding後的標籤 (10000,10)
        y_test_label(ndarray): 測試資料的正確答案 (10000)
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    # 對訓練資料與測試資料的圖像執行常規化
    x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
    x_train, x_test = x_train/255.0, x_test/255.0
    # 將訓練資料與測試資料的標籤轉換為以One-hot encoding來呈現10的類別
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    return x_train, x_test, y_train, y_test


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers

def make_convlayer():
    """ 
    建立模型
    """
    # Sequential物件
    model = Sequential()
    # 卷積層 1
    model.add(Conv2D(filters=64, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu', 
                     input_shape=(32,32,3)))
    # 2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 卷積層 2
    model.add(Conv2D(filters=128, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu'))
    # 2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 卷積層 3
    model.add(Conv2D(filters=256, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu')) 
    #2 × 2 池化層 
    model.add(MaxPooling2D(pool_size=2)) 
    # 扁平層 
    model.add(Flatten()) 
    # 丟棄法
    model.add(Dropout(0.4))
    # 第7層
    model.add(Dense(512, activation='relu'))
    # 輸出層
    model.add(Dense(10, activation='softmax'))

    # 優化器為 Adam
    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=["accuracy"])
    return model

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend

class CyclicLR(Callback):
    """
    Attributes:
        lr_min(float) : 學習率下限
        lr_max(float) : 學習率上限
        step_size(int) : 步長
        mode(str) : 學習率規劃選擇
        gamma(float) : 指數衰減率
        scale_fn(function) : 以lambda定義Scaling函數
        clr_iterations(float) : 迭代次數 (視為循環性學習率的實施次數)
        trn_iterations(float) : 迭代次數 (作為批次的反覆次數)
        scale_mode(int) : 學習率規劃候選
            # 0: 三角學習率 
            # 1: 三角學習率，學習率減半 
            # 2: 三角學習率，學習率指數衰減 
    """ 
    
    def __init__(self, lr_min, lr_max, step_size, mode, gamma=0.99994): 
        """ 
        Parameters: 
            lr_min(float) : 學習率下限 
            lr_max(float) : 學習率上限
            step_size(int) : 步長
            mode(str) : 學習率規劃選擇
            gamma(float) : 指數衰減率
        """
        self.lr_min = lr_min       # 學習率下限
        self.lr_max = lr_max       # 學習率上限
        self.step_size = step_size # 步長 
        self.mode = mode           # 學習率規劃選擇
        self.gamma = gamma         # 指數衰減率
        self.clr_iterations = 0.   # 循環性學習率的執行次數 
        self.trn_iterations = 0.   # 迭代次數 
        self.history = {}          # 記錄學習率與批次編號
        self._init_scale(gamma)

    def _init_scale(self, gamma):
        """
        學習率的衰減方式
        Parameters:
            gamma(int): 指數衰減率
        """
        # 三角學習率 
        if self.mode == 0:
            self.scale_fn = lambda x: 1.
            self.scale_mode = 'cycle'
        # 三角學習率，學習率減半 
        elif self.mode == 1: 
            self.scale_fn = lambda x: 1 / (2. ** (x - 1)) 
            self.scale_mode = 'cycle' 
        # 三角學習率，學習率指數衰減
        elif self.mode == 2:
            self.scale_fn = lambda x: gamma ** (x)
            self.scale_mode = 'iterations'
    
    def clr(self):
        """
        計算循環性學習率
        設定優化器的初始學習率
        """ 
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size)) 
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        
        if self.scale_mode == 'cycle': # 三角學習率、三角學習率，學習率減半
            decay = np.maximum(0, (1 - x)) * self.scale_fn(cycle)
            return self.lr_min + (self.lr_max - self.lr_min) * decay
        else:                          # 三角學習率，學習率指數衰減
            decay = np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)
            return self.lr_min + (self.lr_max - self.lr_min) * decay

    def on_train_begin(self, logs={}): 
        """
        設定訓練剛開始時所呼出的 優化器初始學習率
        """
        logs = logs or {}
        self.losses = []    # 記錄損失的清單 
        self.lr = []        # 記錄學習率的清單 
        # 設定優化器的初始學習率
        if self.clr_iterations == 0:
            backend.set_value(self.model.optimizer.lr, self.lr_min)
        else:
            backend.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None): 
        """
        當批次結束時就呼出
        設定優化器學習率
        將處理中的批次編號與學習率記錄在history
        """ 
        logs = logs or {} 
        self.trn_iterations += 1 
        self.clr_iterations += 1 
        # 記錄現在的學習率 
        self.history.setdefault('lr', []).append(backend.get_value(self.model.optimizer.lr)) 
        # 記錄現在的迭代數
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        # logs:{'batch': 批次編號 } 的 dict 當中取出金鑰跟數值
        for k, v in logs.items():
            # history: 在{'batch': 批次編號清單} 當中追加現在的批次編號
            self.history.setdefault(k, []).append(v)
        # 執行clr()，設定優化器的學習率
        backend.set_value(self.model.optimizer.lr, self.clr())


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, Callback

def train(x_train, x_test, y_train, y_test, mode=0): 
    """ Parameters: 
        x_train, x_test, y_train, y_test: 訓練以及驗證資料 
        mode(int): 循環性學習率的模式 (0, 1, 2)
    """ 
    batch_size = 128             # 小批次尺寸 
    iteration = 50000            # 資料數
    stepsize = iteration/128 * 4 # 循環步長
    lr_min = 0.0001              # 學習率下限
    lr_max = 0.001               # 學習率上限

    # 建立CyclicLR 物件
    clr_triangular = CyclicLR(mode=mode,          # 學習率規劃選擇
                              lr_min=lr_min,      # 學習率下限
                              lr_max=lr_max,      # 學習率上限
                              step_size=stepsize) # 步長
    # 將CyclicLR物件儲存到清單
    callbacks_list = [clr_triangular]
    # 建立模型
    model = make_convlayer()

    # 資料擴增
    datagen = ImageDataGenerator(width_shift_range=0.1,    # 以圖像寬度的0.1比例隨機橫向移動
                                 height_shift_range=0.1,   # 以圖像高度的0.1比例隨機上下移動
                                 rotation_range=10,        # 在10度的範圍內隨機旋轉
                                 zoom_range=0.1,           # 以0.1的比例隨機放大
                                 horizontal_flip=True)     # 左右翻轉
    # 訓練次數
    epochs = 100
    # 進行訓練
    history = model.fit(
        datagen.flow(x_train, y_train, batch_size=batch_size),
        # 步驟次數是由小批次尺寸除以圖像張數所得出的整數值
        steps_per_epoch=x_train.shape[0] // batch_size,
        epochs=epochs,                    # 學習次數
        verbose=1,                        # 輸出學習進度的狀況
        validation_data=(x_test, y_test), # 驗證資料
        callbacks=callbacks_list)

    return history, clr_triangular


In [ ]:
x_train, x_test, y_train, y_test = prepare_data()

In [ ]:
%%time
history_0, clr_triangular_0 = train(x_train, x_test, y_train, y_test, mode=0)

In [ ]:
%%time
history_1, clr_triangular_1 = train(x_train, x_test, y_train, y_test, mode=1)

In [ ]:
%%time
history_2, clr_triangular_2 = train(x_train, x_test, y_train, y_test, mode=2)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(10, 5)) # 繪圖尺寸
# 繪製訓練資料的精確度
plt.plot(history_0.history['accuracy'], label='CLR', linestyle = '--')
plt.plot(history_1.history['accuracy'], label='CLR, Half Decay', linestyle = '-.')
plt.plot(history_2.history['accuracy'], label='CLR, Exp Decay')
plt.legend()            # 顯示圖例
plt.grid()              # 顯示格線
plt.xlabel('Epoch')     # x 軸標籤
plt.ylabel('Train_Acc') # y 軸標籤
plt.show()


In [ ]:
# 將驗證資料的精確度繪製圖成圖形
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10, 5)) # 繪圖尺寸
# 繪製訓練資料的精確度
plt.plot(history_0.history['val_accuracy'], label='CLR', linestyle = '--')
plt.plot(history_1.history['val_accuracy'], label='CLR, Half Decay', linestyle = '-.')
plt.plot(history_2.history['val_accuracy'], label='CLR, Exp Decay')
plt.legend()            # 顯示圖例
plt.grid()              # 顯示格線
plt.xlabel('Epoch')     # x 軸標籤
plt.ylabel('Val_Acc')   # y 軸標籤
plt.show()


In [ ]:
plt.figure(figsize=(10, 15))# 繪圖尺寸

plt.subplot(3, 1, 1)        # 繪製於 3 × 1格線範圍的1
# 繪製學習率
plt.plot(clr_triangular_0.history['lr'], label='Learning Rate(CLR)')
plt.legend()                # 顯示圖例
plt.grid()                  # 顯示格線
plt.xlabel('Iterations')    # x 軸標籤
plt.ylabel('Learning Rate') # y 軸標籤

plt.subplot(3, 1, 2)        # 繪製於3 × 1格線範圍的2
# 繪製學習率
plt.plot(clr_triangular_1.history['lr'], label='Learning Rate(CLR, Half Decay)')
plt.legend()                # 顯示圖例
plt.grid()                  # 顯示格線
plt.xlabel('Iterations')    # x 軸標籤
plt.ylabel('Learning Rate') # y 軸標籤

plt.subplot(3, 1, 3)        # 繪製於3 × 1格線範圍的3
# 繪製學習率
plt.plot(clr_triangular_2.history['lr'], label='Learning Rate(CLR, Exp Decay)')
plt.legend()                # 顯示圖例
plt.grid()                  # 顯示格線
plt.xlabel('Iterations')    # x 軸標籤
plt.ylabel('Learning Rate') # y 軸標籤
plt.show()
